# Assignment: Real-Time Market Sentiment Analyzer Using LangChain and LangGraph

### Objective
Build a LangChain-powered pipeline (Chain) that:
1. Accepts a company name as input.
2. Extracts or generates its stock code.
3. Uses finance news search tools in LangChain to fetch news about the company.
4. Sends the news to an LLM (Azure OpenAI GPT-4o or GPT-4o-mini) to generate a
structured sentiment profile.
5. Outputs the result as a JSON object.
6. Uses mlflow for tracing, prompt debugging, and monitoring.
Tech Stack & Tools
• Framework: LangChain
• LLM: GPT-4o or GPT-4o-mini (deployed via Azure OpenAI)
• Data Source: GOAT / Yahoo Finance tool / Brave Search / Exa Search in LangChain
• Prompt Management & Observability: mlflow
• Environment: Python (v3.10+ recommended)
Tasks Breakdown
Step 1: Accept Input
• Accept a company name as input (e.g., "Apple Inc").
Step 2: Get Stock Code
• Generate or extract the stock ticker/symbol using:
o Either a static lookup table or an API/tool (like Yahoo Finance OR GOAT Symbol
Suggest).
o Integrate this as the first link in your chain.
Step 3: Fetch Company News
• Use LangChain’s integration with new search tools to fetch the latest news for the
extracted stock symbol.
• Extract a concise list of recent headlines or article summaries.
Step 4: Analyze Sentiment with GPT-4o / GPT-4o-mini
• Pass the news summaries to the GPT-4o model via LangChain with a prompt that asks
the LLM to:
o Classify sentiment.
o Extract named entities: people, places, other companies.
o Provide a structured JSON with the following fields:
{
"company_name": "",
"stock_code": "",
"newsdesc": "",
"sentiment": "Positive/Negative/Neutral",
"people_names": [],
"places_names": [],
"other_companies_referred": [],
"related_industries": [],
"market_implications": "",
"confidence_score": 0.0
}
Tip: Use StructuredOutputParser from LangChain or PydanticOutputParser for JSON formatting.
Step 5: Integrate mlflow
• Log prompts, outputs, and metadata using mlflow.
• Add tracing spans for:
o Stock code extraction.
o News fetching.
o Sentiment parsing.
Deliverables
1. Python Script / Notebook (.py or .ipynb) with:
o Full implementation of the chain.
o LangChain Chain definition.
o Proper use of Azure OpenAI APIs and mlflow.
2. README with:
o Setup instructions.
o Azure and mlflow API configuration steps.
o Sample command to run the chain.
3. Sample Output JSON with a real example for a company like "Microsoft".
Bonus Ideas (Optional)
• Add entity linking for extracted people and companies (e.g., using Wikipedia or
LinkedIn).
• Visualize sentiment trend if historical news is considered.
• Use LangChain's MultiPromptChain to classify and then process different types of
news differently.


# Installation and Library Imports

In [1]:
#!python -m pip install langchain langchain-core langchain-community langchain-experimental --quiet
#!python -m pip install langchain-openai --quiet
#!python -m pip install langchain-exa --quiet
#!python -m pip install -U duckduckgo-search --quiet

In [25]:
import logging
from typing import Optional, Dict, Any

import mlflow
import yfinance as yf
from langchain.chat_models import AzureChatOpenAI
from langchain.tools import tool
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, END
from langchain_community.tools import DuckDuckGoSearchRun
from langgraph.prebuilt import ToolNode
from langchain.tools import StructuredTool
from langchain_exa import ExaSearchRetriever
from typing import TypedDict
import os, requests
import mlflow

### Set llm model and mlflow

In [3]:
from langchain_openai import AzureChatOpenAI
model_name='gpt4o'

model = AzureChatOpenAI(model=model_name)

In [26]:
# use it as a chatmodel - pass a chatprompt
import mlflow
mlflow.set_tracking_uri("http://20.75.92.162:5000/")
mlflow.set_experiment("shubhda-stock-market-analyzer")



<Experiment: artifact_location='mlflow-artifacts:/548600595295884550', creation_time=1758355346062, experiment_id='548600595295884550', last_update_time=1758355346062, lifecycle_stage='active', name='shubhda-stock-market-analyzer', tags={}>

### Set logger

In [27]:
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[logging.StreamHandler()]
)
logger = logging.getLogger(__name__)

### Set Environment Variables

In [28]:
import os
AZURE_OPENAI_DEPLOYMENT = os.getenv("AZURE_OPENAI_DEPLOYMENT", "gpt-4o")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION", "2024-05-01-preview")
if not AZURE_OPENAI_API_KEY or not AZURE_OPENAI_ENDPOINT:
    logger.error("Azure OpenAI credentials are missing. Please set environment variables.")
    raise EnvironmentError("Missing Azure OpenAI API key or endpoint.")

### Functions for getting Stock code and News Search.
### These will take company_name as input and return the Ticker and News related to Company

In [29]:
import requests

STATIC_TICKERS = {
    "Apple Inc": "AAPL",
    "Microsoft": "MSFT",
    "Amazon": "AMZN",
    "Alphabet": "GOOGL",
    "Google": "GOOGL",
    "Tesla": "TSLA",
    "Meta": "META",
    "Nvidia": "NVDA",
}

def get_stock_code(company_name: str) -> str:
    """Resolve company name to stock ticker using static table + Yahoo Finance autocomplete."""
    try:
        # 1. Static lookup
        if company_name in STATIC_TICKERS:
            return STATIC_TICKERS[company_name]

        # 2. Yahoo Finance symbol suggest API
        url = f"https://query1.finance.yahoo.com/v1/finance/search?q={company_name}"
        resp = requests.get(url, timeout=5)
        if resp.status_code == 200:
            data = resp.json()
            if "quotes" in data and data["quotes"]:
                return data["quotes"][0].get("symbol", "UNKNOWN").upper()

        return "UNKNOWN"
    except Exception as e:
        return "UNKNOWN"



def fetch_company_news(company_name: str, stock_code: str = None) -> str:
    """Fetch company news from Brave → Exa → DuckDuckGo, whichever is available."""
    query = f"{company_name} stock market news"

    # 1. Exa Search
    exa_key = os.getenv("EXA_API_KEY")
    if exa_key:
        try:
            from langchain_exa import ExaSearchRetriever
            retriever = ExaSearchRetriever(api_key=exa_key, k=5)  # top 5 docs
            docs = retriever.get_relevant_documents(query)
            if docs:
                headlines = [f"- {doc.page_content}" for doc in docs]
                if headlines:
                    print("[INFO] News provider: Exa")
                    
                    mlflow.log_param("news_provider", "Exa")
                    
                    return "\n".join(headlines)
        except Exception as e:
            print(f"[WARN] Exa search failed: {e}")

    # 2. DuckDuckGo Fallback
    try:
        from langchain_community.tools import DuckDuckGoSearchRun
        search = DuckDuckGoSearchRun()
        results = search.run(query)
        if results:
            mlflow.log_param("news_provider", "DuckDuckGo")
            return results[:2000]
    except Exception as e:
        print(f"[WARN] DuckDuckGo failed: {e}")

    # Final fallback
    return f"No news found for {company_name}"


### Functions for output_parser to display information

In [30]:
# ---------------------------------------------------------
# Sentiment LLM Node
# ---------------------------------------------------------
def response_parser():
    response_schemas = [
        ResponseSchema(name="company_name", description="Name of the company"),
        ResponseSchema(name="stock_code", description="Ticker symbol"),
        ResponseSchema(name="newsdesc", description="Concise summary of news"),
        ResponseSchema(name="sentiment", description="Positive/Negative/Neutral"),
        ResponseSchema(name="people_names", description="List of people"),
        ResponseSchema(name="places_names", description="List of places")
        
    ]
    output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

    return output_parser



In [31]:
parser=response_parser()
format_instructions = parser.get_format_instructions()

### ChatpromptTemplate and llm model


In [32]:
initial_template = """
    You are a financial sentiment analyzer.
    Given the following company ({company_name}, {stock_code}) and its recent news,
    classify sentiment and extract structured entities.

    News:
    {news}

    {format_instructions}
    """

In [33]:
# Register a new prompt
initial_prompt = mlflow.genai.register_prompt(
    name="shubhda-marketsentiment-prompt1",
    template=initial_template,
    # Optional: Provide a commit message to describe the changes
    commit_message="Initial commit",
    # Optional: Set tags applies to the prompt (across versions)
    tags={
        "author": "Shubhda Datta",
        "task": "sentiment-analyser",
        "language": "en",
        'llm': 'gpt-4o'
    },
)

# The prompt object contains information about the registered prompt
print(f"Created prompt '{initial_prompt.name}' (version {initial_prompt.version})")

2025/09/29 12:15:03 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for prompt version to finish creation. Prompt name: shubhda-marketsentiment-prompt1, version 1


Created prompt 'shubhda-marketsentiment-prompt1' (version 1)


In [34]:
new_template = """\
You are a financial sentiment analyzer.
    Given the following company ({company_name}, {stock_code}) and its recent news,
    classify sentiment as positive, negative or neutal and identify structured entities.

    News:
    {news}
    format_instructions:
    {format_instructions}
    """

# Register a new version of an existing prompt
updated_prompt = mlflow.genai.register_prompt(
    name="shubhda-marketsentiment-prompt1",  # Specify the existing prompt name
    template=new_template,
    commit_message="Improvement",
    tags={
        "author": "Shubhda Datta",
    },
)

2025/09/29 12:15:17 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for prompt version to finish creation. Prompt name: shubhda-marketsentiment-prompt1, version 2


In [35]:
# Load the prompt
import mlflow

mlflow_prompt = mlflow.genai.load_prompt("prompts:/shubhda-marketsentiment-prompt1/1") # prompts:/<prompt-name>/<prompt-version>
model = AzureChatOpenAI(
    deployment_name=os.getenv("AZURE_OPENAI_DEPLOYMENT"),  # e.g. "gpt-4o"
    openai_api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    temperature=0,
)


/tmp/ipykernel_3045/2911019598.py:5: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import AzureChatOpenAI``.
  model = AzureChatOpenAI(


In [36]:
mlflow_prompt.template

'\n    You are a financial sentiment analyzer.\n    Given the following company ({company_name}, {stock_code}) and its recent news,\n    classify sentiment and extract structured entities.\n\n    News:\n    {news}\n\n    {format_instructions}\n    '

In [37]:
prompt= ChatPromptTemplate.from_template(mlflow_prompt.template)

In [38]:
prompt

ChatPromptTemplate(input_variables=['company_name', 'format_instructions', 'news', 'stock_code'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['company_name', 'format_instructions', 'news', 'stock_code'], input_types={}, partial_variables={}, template='\n    You are a financial sentiment analyzer.\n    Given the following company ({company_name}, {stock_code}) and its recent news,\n    classify sentiment and extract structured entities.\n\n    News:\n    {news}\n\n    {format_instructions}\n    '), additional_kwargs={})])

### Building Langchain

In [39]:

chain= prompt | model | parser
chain

ChatPromptTemplate(input_variables=['company_name', 'format_instructions', 'news', 'stock_code'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['company_name', 'format_instructions', 'news', 'stock_code'], input_types={}, partial_variables={}, template='\n    You are a financial sentiment analyzer.\n    Given the following company ({company_name}, {stock_code}) and its recent news,\n    classify sentiment and extract structured entities.\n\n    News:\n    {news}\n\n    {format_instructions}\n    '), additional_kwargs={})])
| AzureChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x7d5b97f8d160>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x7d5b977d4ce0>, temperature=0.0, model_kwargs={}, openai_api_key='5435c815c889492d83acfa3493bf02ac', openai_proxy='', azure_endpoint='https://eastus.api.cognitive.microsoft.com/', deployment_name='gpt4o',

### Building Langgraph

In [40]:
# -------------------------------
# Graph Definition
# -------------------------------

def build_graph():
    graph = StateGraph(dict)

    from langchain_core.runnables import RunnableLambda

    # Node 1: Lookup stock ticker
    graph.add_node(
        "stock_lookup",
        RunnableLambda(lambda state: {
            **state,
            "stock_code": get_stock_code(state["company_name"])
        })
    )

    # Node 2: Fetch news
    graph.add_node(
    "news_fetch",
    RunnableLambda(lambda state: {
        **state,
        "news": fetch_company_news(state["company_name"], state.get("stock_code"))
    })
    )

    graph.add_node(
        "sentiment",
        RunnableLambda(lambda state: {
            **state,
            "result": chain.invoke({
                "company_name": state.get("company_name", "UNKNOWN"),
                "stock_code": state.get("stock_code", "UNKNOWN"),
                "news": state.get("news", "No news available"),
                "format_instructions": format_instructions
            })
        })
    )
    

    graph.set_entry_point("stock_lookup")
    graph.add_edge("stock_lookup", "news_fetch")
    graph.add_edge("news_fetch", "sentiment")
    graph.add_edge("sentiment", END)

    return graph.compile()


### Getting sentiment using function and registering with mlflow.

In [41]:
# -------------------------------
# Run
# -------------------------------
import json
def analyze_company_sentiment(company_name: str):
    mlflow.set_experiment("market-sentiment-graph")

    with mlflow.start_run(run_name=company_name):
        graph = build_graph()
        result = graph.invoke({"company_name": company_name})
        mlflow.log_text(json.dumps(result, indent=2), "sentiment.json")
        return result

###  Some sample Output data

In [42]:
out = analyze_company_sentiment("Microsoft")
print(json.dumps(out, indent=2))

[INFO] News provider: Exa


2025-09-29 12:16:59,815 [INFO] HTTP Request: POST https://eastus.api.cognitive.microsoft.com/openai/deployments/gpt4o/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"


🏃 View run Microsoft at: http://20.75.92.162:5000/#/experiments/391555380109070687/runs/765b51cc23eb4af1b6056a3d350aa8ac
🧪 View experiment at: http://20.75.92.162:5000/#/experiments/391555380109070687
{
  "company_name": "Microsoft",
  "stock_code": "MSFT",
  "news": "- Oops, something went wrong\n\n[Skip to navigation](https://finance.yahoo.com/finance.yahoo.com#ybar-navigation) [Skip to main content](https://finance.yahoo.com/finance.yahoo.com#nimbus-app) [Skip to right column](https://finance.yahoo.com/finance.yahoo.com#right-rail)\n\n# Microsoft Corp. (MSFT) Rallied in Q2, Driven by Azure growth\n\n[Soumya Eswaran](https://finance.yahoo.com/author/)\n\nMon, September 22, 2025 at 6:48 AM MDT3 min read\n\n- [StockStory Top Pick](https://stockstory.org/high-quality/top-6-to-buy-this-week?partner=yahoo&utm_source=yahoo&utm_medium=content_ticker&utm_campaign=contenthighqualitystock)\n\n\n\n[MSFT\\\n -0.57%](https://finance.yahoo.com/quote/MSFT/)\n\n\n[Macquarie Asset Management](https:/

In [43]:
out = analyze_company_sentiment("Amazon")
print(json.dumps(out, indent=2))

[INFO] News provider: Exa


2025-09-29 12:17:23,811 [INFO] HTTP Request: POST https://eastus.api.cognitive.microsoft.com/openai/deployments/gpt4o/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"


🏃 View run Amazon at: http://20.75.92.162:5000/#/experiments/391555380109070687/runs/e15de4cce1d5467cb3d4a902ad6b1028
🧪 View experiment at: http://20.75.92.162:5000/#/experiments/391555380109070687
{
  "company_name": "Amazon",
  "stock_code": "AMZN",
  "news": "- \n \nThis story appeared on marketwatch.com, 2025-09-23 22:31:48.762000.\n \n\n- \nThis story appeared on marketwatch.com, 2025-09-23 21:17:38.\n\n- \n \n \nAmazon.com Inc. (NASDAQ: AMZN) has been one of the stock market\u2019s biggest success stories ever. The company had its initial public offering in May 1997 and traded for an astonishingly low split-adjusted price of just seven cents per share. Since then, the stock has gained over 308,200%\u2026\n \n \nThis story appeared on 247wallst.com, 2025-09-19 12:20:58.\n \n\n- \n \n \nShares of Amazon.com Inc. (NASDAQ: AMZN) lost 5.56% over the past five trading sessions after losing 1.32% the five prior, bringing the stock\u2019s year-to-date gain to just t0.22%. Amazon announce

In [44]:
out = analyze_company_sentiment("Facebook")
print(json.dumps(out, indent=2))

[INFO] News provider: Exa


2025-09-29 12:17:38,912 [INFO] HTTP Request: POST https://eastus.api.cognitive.microsoft.com/openai/deployments/gpt4o/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 429 Too Many Requests"
2025-09-29 12:17:38,913 [INFO] Retrying request to /chat/completions in 19.000000 seconds
2025-09-29 12:18:01,417 [INFO] HTTP Request: POST https://eastus.api.cognitive.microsoft.com/openai/deployments/gpt4o/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"


🏃 View run Facebook at: http://20.75.92.162:5000/#/experiments/391555380109070687/runs/a01da2c41848458aa391cd7bb1c1b543
🧪 View experiment at: http://20.75.92.162:5000/#/experiments/391555380109070687
{
  "company_name": "Facebook",
  "stock_code": "UNKNOWN",
  "news": "- \n \nThis story appeared on barchart.com, 2025-09-25 15:16:51.\n \n\n- \n \n \nThis year, one of the better performers among the Magnificent 7 has been Meta Platforms Inc. (NASDAQ: META). Its shares have outperformed the broader market and are currently up 22.9% in the past six months, and last month they hit an all-time high of $796.25. For comparison, other\u2026\n \n \nThis story appeared on 247wallst.com, 2025-09-25 12:05:44.\n \n\n- [iframe](https://ce31a0688324c4a39b777827ba0dbdf0.safeframe.googlesyndication.com/safeframe/1-0-41/html/container.html)\n\n[Get 100% ad-free experience](https://www.investing.com/ad-free-subscription?source=desktop&medium=takeover_ad)\n\n# Meta CEO Mark Zuckerberg sells $14.19 million 

In [45]:
out = analyze_company_sentiment("Infosys")
print(json.dumps(out, indent=2))

[INFO] News provider: Exa


2025-09-29 12:18:24,743 [INFO] HTTP Request: POST https://eastus.api.cognitive.microsoft.com/openai/deployments/gpt4o/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"


🏃 View run Infosys at: http://20.75.92.162:5000/#/experiments/391555380109070687/runs/dde939d0263a4881a358e4dde7b6cc66
🧪 View experiment at: http://20.75.92.162:5000/#/experiments/391555380109070687
{
  "company_name": "Infosys",
  "stock_code": "UNKNOWN",
  "news": "- NYSE - Delayed Quote\u2022USD\n\n# Infosys Limited (INFY)\n\nFollow\n\n[Compare](https://finance.yahoo.com/compare/INFY)\n\n1D\n5D\n\n### -2.20%\n\n1M\n\n### -4.19%\n\n6M\n\n### 2.06%\n\nYTD\n\n### -1.58%\n\n1Y\n\n### 7.99%\n\n5Y\n\n### 93.85%\n\nAll\n\n### 3,624.42%\n\nKey Events\n\nMountain\n\nLine\n\nCandle\n\nBaseline\n\nMountain\n\nBar\n\n[Advanced Chart](https://finance.yahoo.com/chart/INFY)\n\nLoading Chart for INFY\n\n9/21 12:03 PM\n\nDELL\n\n(left-click to pin tooltip)(right-click to deleteright-click to manage)(long-press to drag)(drag to change anchor time)\n\n| | |\n| --- | --- |\n| Date | |\n| Close | |\n| Open | |\n| High | |\n| Low | |\n| Volume | |\n\n- Previous Close21.97\n- Open21.59\n- Bid21.37 x 3200\

In [ ]:
out = analyze_company_sentiment("Nvidia")
print(json.dumps(out, indent=2))

[INFO] News provider: Exa


2025-09-25 17:03:59,119 [INFO] HTTP Request: POST https://eastus.api.cognitive.microsoft.com/openai/deployments/gpt4o/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 429 Too Many Requests"
2025-09-25 17:03:59,120 [INFO] Retrying request to /chat/completions in 29.000000 seconds
2025-09-25 17:04:32,167 [INFO] HTTP Request: POST https://eastus.api.cognitive.microsoft.com/openai/deployments/gpt4o/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"


🏃 View run Nvidia at: http://20.75.92.162:5000/#/experiments/391555380109070687/runs/a99aee6c4a8a4eb59c117a481e945deb
🧪 View experiment at: http://20.75.92.162:5000/#/experiments/391555380109070687
{
  "company_name": "Nvidia",
  "stock_code": "NVDA",
  "news": "- \n \n \nThe market remains all in on Nvidia (NVDA). Nvidia is the Opening Bid stock of the day, and it's not simply because it continues to be one of the hottest tickers on our platform. Nor is it because Nvidia has revealed key deals with both OpenAI (OPAI.PVT) and Alibaba (BABA) this week. The\u2026\n \n \nThis story appeared on finance.yahoo.com, 2025-09-24 15:59:17.\n \n\n- This copy is for your personal, non-commercial use only. Distribution and use of this material are governed by our Subscriber Agreement and by copyright law. For non-personal use or to order multiple copies, please contact Dow Jones Reprints at 1-800-843-0008 or visit www.djreprints.com.\n- [\u25b2 S&P 500 **+---%** \\|\u25b2 Stock Advisor **+---%** Jo